# DeepWalk

https://arxiv.org/abs/1403.6652

Paper and algorithm authored by Bryan Perozzi, Rami Al-Rfou, Steven Skiena

Implementation by Rob Forgione

In [ ]:
import numpy as np
from abc import ABC

In [ ]:
# class BinaryLogisticRegression(object):
#     def __init__(self, dims, lr):
#         self.dims = dims
#         self.params = np.random.uniform(0., 1., self.dims)
#         self.lr = lr
        
#     def predict(embedding: np.array) -> np.array:
#         y = np.dot(self.params, embedding)
#         return BinaryLogisticRegression._sigmoid(y)
    
#     def parameter_update(self, gradient, lr):
#         self.params = self.params - lr * gradient
    
#     def _sigmoid(self, x):
#         return 1./(1+np.exp(-x))   
    
#     def _d_sigmoid(self, x):
#         u = np.dot(self.params, x)
#         dsigmoid_du = self._sigmoid(u)*(1 - self._sigmoid(u))
#         du_dparams = x * dsigmoid_du
#         self.parameter_update(du_dparams, lr)
#         return self.params * dsigmoid_du

In [ ]:
class Tree(ABC): 
    @staticmethod
    def merge(a, b):
        return InternalNode(a.dims, a, b, None)
    
    @staticmethod
    def build_tree(nodes):
        while len(nodes) > 1:
            nodes = [Tree.merge(nodes[i], nodes[i+1]) for i in range(0, len(nodes) - 1, 2)]
        
    def set_parent(t):
        self.parent = t
        
    def set_left(): self.is_right = False
        
    def set_right(): self.is_right = True

In [ ]:
class InternalNode(Tree):
    def __init__(self, dims, lr, batch_size, left=None, right=None, parent=None, is_right=None):
        self.dims = dims
        self.set_left_child(left, left=True)
        self.set_right_child(right, left=False)
        self.set_parent(parent)
        self.is_right = is_right
        self.params = np.random.uniform(self.dims) 
        self.gradients = []
        self.lr = 0.01
        self.batch_size=64
        
    def set_left_child(child: Tree):
        self.left = child
        self.left.set_parent(self)
        self.left.set_left()
            
    def set_right_child(child: Tree):
        self.right = child
        self.right.set_parent(self)
        self.right.set_right()
            
    def set_parent(parent: Tree):
        self.parent = parent    
    
    def update_gradient(gradient: np.array):
        self.gradients.append(gradient)
        if len(gradients) >= self.batch_size:
            self.params = self.lr * np.stack(self.gradients, axis=0).mean(axis=0)
        self.gradients = []

In [ ]:
class Leaf(Tree):
    def __init__(self, vertex, parent: InternalNode = None, is_right):
        self.parent = parent
        self.is_right = is_right 
        self.vertex = vertex
        
    def update(self, embedding):
        node = self
        gradient = np.random.uniform(size=len(embedding))
        while node.parent is not None:
            is_right = node.is_right
            node = node.parent        
            d = embedding.dot(node.params) if is_right else -embedding.dot(node.params)
            sigma = 1/(1+np.exp(-d)) # if we're a right child, gets p(x=1); else, gets p(x=0)
            u = (1+np.exp(-d))*sigma*(1-sigma)
            gradient += u*node.params
            node.update_gradients(u*embedding)
        gradient += u*node.params
        self.vertex.update_gradients(sum(gradient))

The goal of hierarchical softmax is to make the scoring function run in $O(logv)$ rathern than $O(v)$ by organizing the nodes as a binary tree with a binary classifier at each internal node. At a high level, we follow these steps:
1. We identify a leaf that is contained within the window of our vertex within the current random walk
2. We take that leaf's parent and compute the probability of having followed the correct path (left or right) to the leaf we identified in step 1 by using the model parameters for this internal node combined with the features for the current vertex (which is a row in $\Phi$).
3. We repeat step 2 for all internal nodes until we get to the root
4. The product of all of the internal probabilities gives us the probability of seeing a co-occurrence of the neighbor node given what we know about the node we're exploring
5. $-logPr(u_k|\Phi(v_j))$ is our loss function, where $Pr(u_k|\Phi(v_j))$ is the probability we calculated in step 4
6. We use the loss in step 5 to perform a gradient descent step updating both the parameters of our model and $\Phi(v_j)$:

$$
\theta \leftarrow \theta - \alpha_\theta * \frac{\partial J}{\partial \theta}
\\ 
\Phi \leftarrow \Phi - \alpha_\Phi * \frac{\partial J}{\partial \Phi}
$$

Where $\theta$ represents all of the parameters of all of the models in the internal nodes of the tree, and $\Phi$ represents the latent representation of the current vertex.

In [ ]:
def skipgram(phi, W, window_size):
    for i in range(len(W)):
        v = W[i]
        idx_lower = np.min(i - window_size, 0)
        idx_upper = np.max(i + window_size, len(W))
        neighbors = W[idx_lower:idx_upper] 

In [ ]:
def deepwalk(g, window_size, embedding_size, walks_per_vertex, walk_length):
    phi = np.random.uniform(0., 1., (g.size, embedding_size))
    for i in range(0, walks_per_vertex): 
        shuffled_nodes = g.shuffle().nodes()
        for v in shuffled_nodes:
            W = random_walk(g, v, walk_length)
            skipgram(phi, W, window_size)